**Due Date: Monday, January 22nd, 11:59pm**

- Fill out the missing parts and run the code modules.
- Answer the questions (if any) in a separate document or by adding a new `Text` block inside the Colab.
- Download the solved notebook by going to the menu and clicking `File` > `Download .ipynb`.
- Make sure the downloaded version is showing your solutions.
- Upload your solutions to BruinLearn (under "Colab Assignment #1: Statistical Inference").

In [14]:
!pip install numpy
import numpy as np

np.random.seed(0)

Simulate a dataset of 100 coin flips for a coin with $p$ = P(head) = 0.6.

In [2]:
n = 100
p = 0.6

# TODO: Update y
y = np.random.binomial(1, p, n)
# END OF TODO

y

array([1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0,
       1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1,
       0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1])

## Point Estimation

Estimate the value of $p$ using the data.

In [3]:
def estimator(y):
  # TODO: Update the function
    return np.mean(y)
  # END OF TODO

p_hat = estimator(y)
p_hat

0.62

## Bootstrap

Estimate the standard error of $\hat{p}$ using bootstrap.

In [4]:
def bootstrap_se_est(y, stat_function, B=1000):
  # TODO: Update the function
    t_boot_list = []
    for b in range(B):
        y_boot = np.random.choice(y, len(y), replace = True)
        t_boot = np.mean(y_boot)
        t_boot_list.append(t_boot)
    return np.std(t_boot_list)
  # END OF TODO

print("Standard error of p_hat computed by bootstrap:")
print(bootstrap_se_est(y, estimator))

Standard error of p_hat computed by bootstrap:
0.04889048066853097


Validate the estimated standard error by computing it analytically.

In [5]:
def estimator_analytical_se(p, n):
  # TODO: Update the function
    return(np.sqrt(1/n * p * (1-p)))
  # END OF TODO

print("Analytical standard error for the estimator: ", estimator_analytical_se(p, n))

Analytical standard error for the estimator:  0.048989794855663564


Estimate the 95% confidence interval for $p$.

In [6]:
def confidence_interval_95_for_p(y):
  # TODO: Update the function
    n = len(y)
    p_hat = np.mean(y)
    se_hat = np.sqrt(1/n * p_hat * (1-p_hat))
    ci_lower = p_hat - 1.96*se_hat
    ci_higher = p_hat + 1.96*se_hat
  # END OF TODO
    return (ci_lower, ci_higher)

lower, higher = confidence_interval_95_for_p(y)
print("95% confidence interval for p: ({},{})".format(lower, higher))

95% confidence interval for p: (0.5248642569798291,0.7151357430201709)


Validate the 95% confidence interval for $p$.

In [7]:
ci_contains_p_flags = []
for sim in range(1000):
  # TODO: fill ci_contains_p_flags with the right values
    y = np.random.binomial(1, p, n)
    ci_lower, ci_higher = confidence_interval_95_for_p(y)
    if ci_lower < p and p < ci_higher:
        ci_contains_p_flags.append(1)
    else:
        ci_contains_p_flags.append(0)
  # END OF TODO

coverage = np.mean(ci_contains_p_flags)
print("Coverage of 95% confidence interval for p: ", coverage)

Coverage of 95% confidence interval for p:  0.96


## Bayesian Inference

**[Optional]**

Estimate $p$ using Bayesian inference. As the prior for $p$ use Normal(0.5, 0.1).

In [8]:
!pip install Cython
!pip install pystan

In [13]:
import pystan

In [10]:
import multiprocessing
multiprocessing.set_start_method("fork")

In [11]:
model_code = """
data {
  int<lower=0> n;
  int<lower=0,upper=1> y[n];
}
parameters {
  real<lower=0,upper=1> p;
}
model {
  p ~ normal(0.5, 0.1);
  for (i in 1:n)
    y[i] ~ bernoulli(p);
}
"""

model = pystan.StanModel(model_code=model_code)
fit = model.sampling(data={"n": n, "y": y}, iter=2000, chains=4, n_jobs=4)
print(fit.stansummary())

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_da99211d8d54f3aa1eb8363291e9586e NOW.
In file included from /var/folders/m2/xhnjn3lj1s7fyxc78nrtt7gc0000gn/T/pystan_o4eqby4z/stanfit4anon_model_da99211d8d54f3aa1eb8363291e9586e_5231260145131113191.cpp:1280:
In file included from /Users/laurenmizner/anaconda3/lib/python3.11/site-packages/numpy/core/include/numpy/arrayobject.h:5:
In file included from /Users/laurenmizner/anaconda3/lib/python3.11/site-packages/numpy/core/include/numpy/ndarrayobject.h:12:
In file included from /Users/laurenmizner/anaconda3/lib/python3.11/site-packages/numpy/core/include/numpy/ndarraytypes.h:1940:
/Users/laurenmizner/anaconda3/lib/python3.11/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: "Using deprecated NumPy API, disable it with "          "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-W#warnings]
#warning "Using deprecated NumPy API, disable it with " \
 ^
In file included from /var/folders/m2/xhnjn3lj1s7fyxc7

In file included from /var/folders/m2/xhnjn3lj1s7fyxc78nrtt7gc0000gn/T/pystan_o4eqby4z/stanfit4anon_model_da99211d8d54f3aa1eb8363291e9586e_5231260145131113191.cpp:1286:
In file included from /Users/laurenmizner/anaconda3/lib/python3.11/site-packages/pystan/py_var_context.hpp:12:
In file included from /Users/laurenmizner/anaconda3/lib/python3.11/site-packages/pystan/stan/src/stan/io/dump.hpp:6:
In file included from /Users/laurenmizner/anaconda3/lib/python3.11/site-packages/pystan/stan/lib/stan_math/stan/math/prim/mat.hpp:344:
In file included from /Users/laurenmizner/anaconda3/lib/python3.11/site-packages/pystan/stan/lib/stan_math/stan/math/prim/arr.hpp:46:
In file included from /Users/laurenmizner/anaconda3/lib/python3.11/site-packages/pystan/stan/lib/stan_math/stan/math/prim/arr/functor/integrate_ode_rk45.hpp:17:
In file included from /Users/laurenmizner/anaconda3/lib/python3.11/site-packages/pystan/stan/lib/stan_math/lib/boost_1.69.0/boost/numeric/odeint.hpp:37:
In file included fro

In file included from /var/folders/m2/xhnjn3lj1s7fyxc78nrtt7gc0000gn/T/pystan_o4eqby4z/stanfit4anon_model_da99211d8d54f3aa1eb8363291e9586e_5231260145131113191.cpp:1287:
/Users/laurenmizner/anaconda3/lib/python3.11/site-packages/pystan/stan_fit.hpp:841:12: warning: 'auto_ptr<stan::io::var_context>' is deprecated [-Wdeprecated-declarations]
      std::auto_ptr<stan::io::var_context> init_context_ptr;
           ^
/Library/Developer/CommandLineTools/SDKs/MacOSX.sdk/usr/include/c++/v1/__memory/auto_ptr.h:30:28: note: 'auto_ptr<stan::io::var_context>' has been explicitly marked deprecated here
class _LIBCPP_TEMPLATE_VIS _LIBCPP_DEPRECATED_IN_CXX11 auto_ptr
                           ^
/Library/Developer/CommandLineTools/SDKs/MacOSX.sdk/usr/include/c++/v1/__config:825:41: note: expanded from macro '_LIBCPP_DEPRECATED_IN_CXX11'
#    define _LIBCPP_DEPRECATED_IN_CXX11 _LIBCPP_DEPRECATED
                                        ^
/Library/Developer/CommandLineTools/SDKs/MacOSX.sdk/usr/include/c+

In file included from /var/folders/m2/xhnjn3lj1s7fyxc78nrtt7gc0000gn/T/pystan_o4eqby4z/stanfit4anon_model_da99211d8d54f3aa1eb8363291e9586e_5231260145131113191.cpp:1286:
In file included from /Users/laurenmizner/anaconda3/lib/python3.11/site-packages/pystan/py_var_context.hpp:12:
In file included from /Users/laurenmizner/anaconda3/lib/python3.11/site-packages/pystan/stan/src/stan/io/dump.hpp:6:
In file included from /Users/laurenmizner/anaconda3/lib/python3.11/site-packages/pystan/stan/lib/stan_math/stan/math/prim/mat.hpp:344:
In file included from /Users/laurenmizner/anaconda3/lib/python3.11/site-packages/pystan/stan/lib/stan_math/stan/math/prim/arr.hpp:46:
In file included from /Users/laurenmizner/anaconda3/lib/python3.11/site-packages/pystan/stan/lib/stan_math/stan/math/prim/arr/functor/integrate_ode_rk45.hpp:17:
In file included from /Users/laurenmizner/anaconda3/lib/python3.11/site-packages/pystan/stan/lib/stan_math/lib/boost_1.69.0/boost/numeric/odeint.hpp:73:
In file included fro

In file included from /var/folders/m2/xhnjn3lj1s7fyxc78nrtt7gc0000gn/T/pystan_o4eqby4z/stanfit4anon_model_da99211d8d54f3aa1eb8363291e9586e_5231260145131113191.cpp:1286:
In file included from /Users/laurenmizner/anaconda3/lib/python3.11/site-packages/pystan/py_var_context.hpp:12:
In file included from /Users/laurenmizner/anaconda3/lib/python3.11/site-packages/pystan/stan/src/stan/io/dump.hpp:6:
In file included from /Users/laurenmizner/anaconda3/lib/python3.11/site-packages/pystan/stan/lib/stan_math/stan/math/prim/mat.hpp:9:
In file included from /Users/laurenmizner/anaconda3/lib/python3.11/site-packages/pystan/stan/lib/stan_math/stan/math/prim/mat/meta/as_array_or_scalar.hpp:4:
In file included from /Users/laurenmizner/anaconda3/lib/python3.11/site-packages/pystan/stan/lib/stan_math/lib/eigen_3.3.3/Eigen/Dense:1:
In file included from /Users/laurenmizner/anaconda3/lib/python3.11/site-packages/pystan/stan/lib/stan_math/lib/eigen_3.3.3/Eigen/Core:420:
/Users/laurenmizner/anaconda3/lib/py

In file included from /var/folders/m2/xhnjn3lj1s7fyxc78nrtt7gc0000gn/T/pystan_o4eqby4z/stanfit4anon_model_da99211d8d54f3aa1eb8363291e9586e_5231260145131113191.cpp:1286:
In file included from /Users/laurenmizner/anaconda3/lib/python3.11/site-packages/pystan/py_var_context.hpp:12:
In file included from /Users/laurenmizner/anaconda3/lib/python3.11/site-packages/pystan/stan/src/stan/io/dump.hpp:6:
In file included from /Users/laurenmizner/anaconda3/lib/python3.11/site-packages/pystan/stan/lib/stan_math/stan/math/prim/mat.hpp:9:
In file included from /Users/laurenmizner/anaconda3/lib/python3.11/site-packages/pystan/stan/lib/stan_math/stan/math/prim/mat/meta/as_array_or_scalar.hpp:4:
In file included from /Users/laurenmizner/anaconda3/lib/python3.11/site-packages/pystan/stan/lib/stan_math/lib/eigen_3.3.3/Eigen/Dense:1:
In file included from /Users/laurenmizner/anaconda3/lib/python3.11/site-packages/pystan/stan/lib/stan_math/lib/eigen_3.3.3/Eigen/Core:420:
/Users/laurenmizner/anaconda3/lib/py

In file included from /var/folders/m2/xhnjn3lj1s7fyxc78nrtt7gc0000gn/T/pystan_o4eqby4z/stanfit4anon_model_da99211d8d54f3aa1eb8363291e9586e_5231260145131113191.cpp:1286:
In file included from /Users/laurenmizner/anaconda3/lib/python3.11/site-packages/pystan/py_var_context.hpp:12:
In file included from /Users/laurenmizner/anaconda3/lib/python3.11/site-packages/pystan/stan/src/stan/io/dump.hpp:6:
In file included from /Users/laurenmizner/anaconda3/lib/python3.11/site-packages/pystan/stan/lib/stan_math/stan/math/prim/mat.hpp:9:
In file included from /Users/laurenmizner/anaconda3/lib/python3.11/site-packages/pystan/stan/lib/stan_math/stan/math/prim/mat/meta/as_array_or_scalar.hpp:4:
In file included from /Users/laurenmizner/anaconda3/lib/python3.11/site-packages/pystan/stan/lib/stan_math/lib/eigen_3.3.3/Eigen/Dense:1:
In file included from /Users/laurenmizner/anaconda3/lib/python3.11/site-packages/pystan/stan/lib/stan_math/lib/eigen_3.3.3/Eigen/Core:420:
/Users/laurenmizner/anaconda3/lib/py

185 warnings generated.




Gradient evaluation took 2.2e-05 seconds
Gradient evaluation took 2.3e-05 seconds
1000 transitions using 10 leapfrog steps per transition would take 0.23 seconds.
Adjust your expectations accordingly!


1000 transitions using 10 leapfrog steps per transition would take 0.22 seconds.
Adjust your expectations accordingly!



Gradient evaluation took 3.6e-05 seconds
1000 transitions using 10 leapfrog steps per transition would take 0.36 seconds.
Adjust your expectations accordingly!


Iteration:    1 / 2000 [  0%]  (Warmup)
Iteration:    1 / 2000 [  0%]  (Warmup)
Iteration:    1 / 2000 [  0%]  (Warmup)

Gradient evaluation took 1.8e-05 seconds
1000 transitions using 10 leapfrog steps per transition would take 0.18 seconds.
Adjust your expectations accordingly!


Iteration:    1 / 2000 [  0%]  (Warmup)
Iteration:  200 / 2000 [ 10%]  (Warmup)
Iteration:  200 / 2000 [ 10%]  (Warmup)
Iteration:  200 / 2000 [ 10%]  (Warmup)
Iteration:  200 / 2000 [ 10%]  (Warmup)
Iteration:  400 / 2000 [ 20%

Compute the Bayesian inference results if our data contains 20 coin tosses instead.

In [12]:
n = 20
p = 0.6
y = np.random.binomial(1, p, n)

model = pystan.StanModel(model_code=model_code)
fit = model.sampling(data={"n": n, "y": y}, iter=2000, chains=4, n_jobs=4)
print(fit.stansummary())

In file included from /var/folders/m2/xhnjn3lj1s7fyxc78nrtt7gc0000gn/T/pystan_psvy0bds/stanfit4anon_model_da99211d8d54f3aa1eb8363291e9586e_5280017692911393952.cpp:1280:
In file included from /Users/laurenmizner/anaconda3/lib/python3.11/site-packages/numpy/core/include/numpy/arrayobject.h:5:
In file included from /Users/laurenmizner/anaconda3/lib/python3.11/site-packages/numpy/core/include/numpy/ndarrayobject.h:12:
In file included from /Users/laurenmizner/anaconda3/lib/python3.11/site-packages/numpy/core/include/numpy/ndarraytypes.h:1940:
/Users/laurenmizner/anaconda3/lib/python3.11/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: "Using deprecated NumPy API, disable it with "          "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-W#warnings]
#warning "Using deprecated NumPy API, disable it with " \
 ^
In file included from /var/folders/m2/xhnjn3lj1s7fyxc78nrtt7gc0000gn/T/pystan_psvy0bds/stanfit4anon_model_da99211d8d54f3aa1eb8363291e9586e_528001769

In file included from /var/folders/m2/xhnjn3lj1s7fyxc78nrtt7gc0000gn/T/pystan_psvy0bds/stanfit4anon_model_da99211d8d54f3aa1eb8363291e9586e_5280017692911393952.cpp:1286:
In file included from /Users/laurenmizner/anaconda3/lib/python3.11/site-packages/pystan/py_var_context.hpp:12:
In file included from /Users/laurenmizner/anaconda3/lib/python3.11/site-packages/pystan/stan/src/stan/io/dump.hpp:6:
In file included from /Users/laurenmizner/anaconda3/lib/python3.11/site-packages/pystan/stan/lib/stan_math/stan/math/prim/mat.hpp:344:
In file included from /Users/laurenmizner/anaconda3/lib/python3.11/site-packages/pystan/stan/lib/stan_math/stan/math/prim/arr.hpp:46:
In file included from /Users/laurenmizner/anaconda3/lib/python3.11/site-packages/pystan/stan/lib/stan_math/stan/math/prim/arr/functor/integrate_ode_rk45.hpp:17:
In file included from /Users/laurenmizner/anaconda3/lib/python3.11/site-packages/pystan/stan/lib/stan_math/lib/boost_1.69.0/boost/numeric/odeint.hpp:37:
In file included fro

In file included from /var/folders/m2/xhnjn3lj1s7fyxc78nrtt7gc0000gn/T/pystan_psvy0bds/stanfit4anon_model_da99211d8d54f3aa1eb8363291e9586e_5280017692911393952.cpp:1287:
/Users/laurenmizner/anaconda3/lib/python3.11/site-packages/pystan/stan_fit.hpp:841:12: warning: 'auto_ptr<stan::io::var_context>' is deprecated [-Wdeprecated-declarations]
      std::auto_ptr<stan::io::var_context> init_context_ptr;
           ^
/Library/Developer/CommandLineTools/SDKs/MacOSX.sdk/usr/include/c++/v1/__memory/auto_ptr.h:30:28: note: 'auto_ptr<stan::io::var_context>' has been explicitly marked deprecated here
class _LIBCPP_TEMPLATE_VIS _LIBCPP_DEPRECATED_IN_CXX11 auto_ptr
                           ^
/Library/Developer/CommandLineTools/SDKs/MacOSX.sdk/usr/include/c++/v1/__config:825:41: note: expanded from macro '_LIBCPP_DEPRECATED_IN_CXX11'
#    define _LIBCPP_DEPRECATED_IN_CXX11 _LIBCPP_DEPRECATED
                                        ^
/Library/Developer/CommandLineTools/SDKs/MacOSX.sdk/usr/include/c+

In file included from /var/folders/m2/xhnjn3lj1s7fyxc78nrtt7gc0000gn/T/pystan_psvy0bds/stanfit4anon_model_da99211d8d54f3aa1eb8363291e9586e_5280017692911393952.cpp:1286:
In file included from /Users/laurenmizner/anaconda3/lib/python3.11/site-packages/pystan/py_var_context.hpp:12:
In file included from /Users/laurenmizner/anaconda3/lib/python3.11/site-packages/pystan/stan/src/stan/io/dump.hpp:6:
In file included from /Users/laurenmizner/anaconda3/lib/python3.11/site-packages/pystan/stan/lib/stan_math/stan/math/prim/mat.hpp:344:
In file included from /Users/laurenmizner/anaconda3/lib/python3.11/site-packages/pystan/stan/lib/stan_math/stan/math/prim/arr.hpp:46:
In file included from /Users/laurenmizner/anaconda3/lib/python3.11/site-packages/pystan/stan/lib/stan_math/stan/math/prim/arr/functor/integrate_ode_rk45.hpp:17:
In file included from /Users/laurenmizner/anaconda3/lib/python3.11/site-packages/pystan/stan/lib/stan_math/lib/boost_1.69.0/boost/numeric/odeint.hpp:73:
In file included fro

In file included from /var/folders/m2/xhnjn3lj1s7fyxc78nrtt7gc0000gn/T/pystan_psvy0bds/stanfit4anon_model_da99211d8d54f3aa1eb8363291e9586e_5280017692911393952.cpp:1286:
In file included from /Users/laurenmizner/anaconda3/lib/python3.11/site-packages/pystan/py_var_context.hpp:12:
In file included from /Users/laurenmizner/anaconda3/lib/python3.11/site-packages/pystan/stan/src/stan/io/dump.hpp:6:
In file included from /Users/laurenmizner/anaconda3/lib/python3.11/site-packages/pystan/stan/lib/stan_math/stan/math/prim/mat.hpp:9:
In file included from /Users/laurenmizner/anaconda3/lib/python3.11/site-packages/pystan/stan/lib/stan_math/stan/math/prim/mat/meta/as_array_or_scalar.hpp:4:
In file included from /Users/laurenmizner/anaconda3/lib/python3.11/site-packages/pystan/stan/lib/stan_math/lib/eigen_3.3.3/Eigen/Dense:1:
In file included from /Users/laurenmizner/anaconda3/lib/python3.11/site-packages/pystan/stan/lib/stan_math/lib/eigen_3.3.3/Eigen/Core:420:
/Users/laurenmizner/anaconda3/lib/py

In file included from /var/folders/m2/xhnjn3lj1s7fyxc78nrtt7gc0000gn/T/pystan_psvy0bds/stanfit4anon_model_da99211d8d54f3aa1eb8363291e9586e_5280017692911393952.cpp:1286:
In file included from /Users/laurenmizner/anaconda3/lib/python3.11/site-packages/pystan/py_var_context.hpp:12:
In file included from /Users/laurenmizner/anaconda3/lib/python3.11/site-packages/pystan/stan/src/stan/io/dump.hpp:6:
In file included from /Users/laurenmizner/anaconda3/lib/python3.11/site-packages/pystan/stan/lib/stan_math/stan/math/prim/mat.hpp:9:
In file included from /Users/laurenmizner/anaconda3/lib/python3.11/site-packages/pystan/stan/lib/stan_math/stan/math/prim/mat/meta/as_array_or_scalar.hpp:4:
In file included from /Users/laurenmizner/anaconda3/lib/python3.11/site-packages/pystan/stan/lib/stan_math/lib/eigen_3.3.3/Eigen/Dense:1:
In file included from /Users/laurenmizner/anaconda3/lib/python3.11/site-packages/pystan/stan/lib/stan_math/lib/eigen_3.3.3/Eigen/Core:420:
/Users/laurenmizner/anaconda3/lib/py

In file included from /var/folders/m2/xhnjn3lj1s7fyxc78nrtt7gc0000gn/T/pystan_psvy0bds/stanfit4anon_model_da99211d8d54f3aa1eb8363291e9586e_5280017692911393952.cpp:1286:
In file included from /Users/laurenmizner/anaconda3/lib/python3.11/site-packages/pystan/py_var_context.hpp:12:
In file included from /Users/laurenmizner/anaconda3/lib/python3.11/site-packages/pystan/stan/src/stan/io/dump.hpp:6:
In file included from /Users/laurenmizner/anaconda3/lib/python3.11/site-packages/pystan/stan/lib/stan_math/stan/math/prim/mat.hpp:9:
In file included from /Users/laurenmizner/anaconda3/lib/python3.11/site-packages/pystan/stan/lib/stan_math/stan/math/prim/mat/meta/as_array_or_scalar.hpp:4:
In file included from /Users/laurenmizner/anaconda3/lib/python3.11/site-packages/pystan/stan/lib/stan_math/lib/eigen_3.3.3/Eigen/Dense:1:
In file included from /Users/laurenmizner/anaconda3/lib/python3.11/site-packages/pystan/stan/lib/stan_math/lib/eigen_3.3.3/Eigen/Core:420:
/Users/laurenmizner/anaconda3/lib/py

185 warnings generated.




Gradient evaluation took 1.8e-05 seconds
Gradient evaluation took 2.2e-05 seconds

1000 transitions using 10 leapfrog steps per transition would take 0.18 seconds.
Adjust your expectations accordingly!


1000 transitions using 10 leapfrog steps per transition would take 0.22 seconds.
Gradient evaluation took 1.8e-05 seconds
Adjust your expectations accordingly!


1000 transitions using 10 leapfrog steps per transition would take 0.18 seconds.
Adjust your expectations accordingly!


Iteration:    1 / 2000 [  0%]  (Warmup)
Iteration:    1 / 2000 [  0%]  (Warmup)
Iteration:    1 / 2000 [  0%]  (Warmup)

Gradient evaluation took 1.7e-05 seconds
1000 transitions using 10 leapfrog steps per transition would take 0.17 seconds.
Adjust your expectations accordingly!


Iteration:    1 / 2000 [  0%]  (Warmup)
Iteration:  200 / 2000 [ 10%]  (Warmup)
Iteration:  200 / 2000 [ 10%]  (Warmup)
Iteration:  200 / 2000 [ 10%]  (Warmup)
Iteration:  400 / 2000 [ 20%]  (Warmup)
Iteration:  400 / 2000 [ 20%